In [75]:
import time
import numpy
import urllib
import scipy.optimize
import random
from collections import defaultdict
from collections import Counter
import string
from sklearn import linear_model
import ast
import importlib
import numpy as np
from scipy.spatial import distance
import random
import nltk
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from nltk.corpus import stopwords
nltk.download("averaged_perceptron_tagger")
nltk.download('stopwords')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Charlie/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Charlie/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [100]:
import as2_analysis_utils as as2_analysis

importlib.reload(as2_analysis)

<module 'as2_analysis_utils' from '/Users/Charlie/Documents/Work_School/UCSD/Grad/Fall_2020/CSE258/assignment2/as2_analysis_utils.py'>

In [101]:
data_all = list(as2_analysis.parseData_line("renttherunway_final_data.json"))

In [102]:
def extract_features(data_all, key):
    """
    return data of particualr key
    """
    return [d[key] for d in data_all]

In [109]:
stop_words = [word for word in stopwords.words("english") 
              if word != "not" and 
              not ("n'" in word)]

 # Count word frequency

In [110]:
# top words in fit reviews
fit_reviews_all = [
    d["review_text"] + " " + d["review_summary"] for d in data_all if d["fit"] == "fit"
]
small_reviews_all = [
    d["review_text"] + " " + d["review_summary"]
    for d in data_all
    if d["fit"] == "small"
]
large_reviews_all = [
    d["review_text"] + " " + d["review_summary"]
    for d in data_all
    if d["fit"] == "large"
]



In [312]:
resample_size = np.mean([len(small_reviews_all),len(large_reviews_all)])

In [315]:
fit_reviews_all_resample = random.sample(fit_reviews_all,int(resample_size))

In [316]:
def tokenize_data_sets(dataset, n=1):
    """
    Each item of dataset is a str
    """
    return [as2_analysis.tokenize_paragraph(d, n=n,remove_stopwrods=True,stopwords=stop_words) for d in dataset]



In [317]:
# tokenize dataset
fit_tokenized_all = tokenize_data_sets(fit_reviews_all_resample,n=2)
small_tokenized_all = tokenize_data_sets(small_reviews_all,n=2)
large_tokenized_all = tokenize_data_sets(large_reviews_all,n=2)



In [318]:
fit_word_count = as2_analysis.count_word_freq(fit_tokenized_all)
small_word_count = as2_analysis.count_word_freq(small_tokenized_all)
large_word_count = as2_analysis.count_word_freq(large_tokenized_all)


 # Get most frequent adjective

In [177]:
def get_adj(word_list):
    """
    return adj from a list of words 
    """
    word_list_adj = []
    
    for word in word_list:
        if isinstance(word,str) and nltk.pos_tag([word])[0][1][0] == "J":
            word_list_adj.append(word)
    
    return word_list_adj

In [178]:
def top_n_adj(word_count_dict):
    """
    only works for unigram
    """
    adj_list = get_adj(word_count_dict.keys())
    adj_count = {adj : word_count_dict[adj] for adj in adj_list}
    
    # sort them into pairs
    adj_count_pair = [(count, key) for key, count in adj_count.items()]

    # sort
    adj_count_pair.sort(reverse=True)

    return adj_count_pair

In [179]:
start = time.time()
top_small_adj = top_n_adj(small_word_count)
top_fit_adj = top_n_adj(fit_word_count)
top_big_adj = top_n_adj(large_word_count)


end = time.time()
print(f"time consume: {end-start}s")

time consume: 15.15400218963623s


In [180]:
top_small_adj[:20]

[(11137, 'small'),
 (9126, 'great'),
 (8424, 'little'),
 (5654, 'comfortable'),
 (4705, 'short'),
 (3386, 'many'),
 (2854, 'gorgeous'),
 (2675, 'overall'),
 (2623, 'much'),
 (2250, 'black'),
 (1975, 'nice'),
 (1867, 'larger'),
 (1847, 'large'),
 (1826, 'good'),
 (1764, 'able'),
 (1674, 'big'),
 (1268, 'formal'),
 (1171, 'smaller'),
 (1101, 'high'),
 (979, 'best')]

In [181]:
top_fit_adj[:20]

[(60010, 'great'),
 (48689, 'comfortable'),
 (39921, 'little'),
 (22857, 'many'),
 (18916, 'true'),
 (17789, 'short'),
 (15455, 'black'),
 (13260, 'gorgeous'),
 (12799, 'much'),
 (11649, 'nice'),
 (11565, 'overall'),
 (10758, 'small'),
 (10152, 'good'),
 (10116, 'easy'),
 (8865, 'big'),
 (8315, 'formal'),
 (7258, 'able'),
 (6230, 'high'),
 (6080, 'large'),
 (5969, 'regular')]

In [209]:
top_big_adj[:500]

[(9276, 'great'),
 (8972, 'little'),
 (8489, 'large'),
 (7584, 'comfortable'),
 (7517, 'big'),
 (3201, 'small'),
 (3011, 'many'),
 (2654, 'much'),
 (2602, 'overall'),
 (2406, 'smaller'),
 (2383, 'short'),
 (2371, 'loose'),
 (2246, 'nice'),
 (2214, 'gorgeous'),
 (2120, 'black'),
 (1793, 'good'),
 (1615, 'easy'),
 (1395, 'able'),
 (1199, 'larger'),
 (1146, 'formal'),
 (1081, 'low'),
 (1020, 'regular'),
 (1019, 'heavy'),
 (947, 'best'),
 (930, 'high'),
 (913, 'extra'),
 (889, 'casual'),
 (785, 'red'),
 (742, 'huge'),
 (739, 'different'),
 (695, 'normal'),
 (676, 'true'),
 (650, 'available'),
 (644, 'happy'),
 (601, 'bigger'),
 (590, 'white'),
 (576, 'next'),
 (571, 'hot'),
 (567, 'ive'),
 (534, 'due'),
 (481, 'full'),
 (465, 'classic'),
 (464, 'worried'),
 (462, 'open'),
 (456, 'soft'),
 (448, 'new'),
 (441, 'wide'),
 (429, 'hard'),
 (423, 'fabulous'),
 (418, 'chic'),
 (408, 'conservative'),
 (356, 'second'),
 (353, 'usual'),
 (347, 'fantastic'),
 (338, 'bad'),
 (319, 'uncomfortable'),
 (

 Try bigram and trigram

In [159]:
def top_n_adj_n_gram(word_count_dict):

    ## get the most common ajectives
    adj_count = {
        key: word_count_dict[key]
        for key in word_count_dict
        # if either of the word is an adj, preserve
        if any(
            wordType[0] == "J"
            for _, wordType in nltk.pos_tag(key if type(key) == tuple else (key,))
        )
    }

    # sort them into pairs
    adj_count_pair = list(adj_count.items())

    # # sort
    adj_count_pair.sort(reverse=True, key=lambda x: x[1])

    return adj_count_pair



In [187]:
def top_adj_pipeline(fit_word_count, small_word_count, large_word_count):

#     fit_tokenized_all = tokenize_data_sets(fit_reviews_all, n)
#     small_tokenized_all = tokenize_data_sets(small_reviews_all, n)
#     large_tokenized_all = tokenize_data_sets(large_reviews_all, n)

#     fit_word_count = as2_analysis.count_word_freq(fit_tokenized_all)
#     small_word_count = as2_analysis.count_word_freq(small_tokenized_all)
#     large_word_count = as2_analysis.count_word_freq(large_tokenized_all)

    start = time.time()
    top_fit_grams = top_n_adj_n_gram(fit_word_count)
    top_small_grams = top_n_adj_n_gram(small_word_count)
    top_large_grams = top_n_adj_n_gram(large_word_count)
    end = time.time()
    print(f"time consume: {end-start}s")
    return top_fit_grams, top_small_grams, top_large_grams



In [319]:
top_fit_grams, top_small_grams, top_large_grams = top_adj_pipeline(fit_word_count,
                                                                   small_word_count,
                                                                   large_word_count)

time consume: 206.87134909629822s


In [320]:
bigrams_common = as2_analysis.count_word_freq(fit_tokenized_all + small_tokenized_all + large_tokenized_all)

In [321]:
def intersection_fit_small_large(fit_top, small_top, large_top, world_count_all,index=0):
    """
    Find the intersection between top ngrams from each set
    """
    fit_ngram = [item[index] for item in fit_top]
    small_ngram = [item[index] for item in small_top]
    large_ngram = [item[index] for item in large_top]
    
    intersection = set.intersection(set(fit_ngram),set(small_ngram),set(large_ngram))
    return_tuple = [(world_count_all[item],item) for item in intersection]
    return_tuple.sort(reverse=True)
#     return return_tuple.sort(reverse=True)
    return fit_ngram, small_ngram, large_ngram, return_tuple

In [322]:
fit_ngram, small_ngram, large_ngram,top_common_ngram = intersection_fit_small_large(top_fit_grams[:500], 
                                                top_small_grams[:500], 
                                                top_large_grams[:500], 
                                                bigrams_common,
                                                0)

In [334]:
len(top_fit_grams)

71739

In [333]:
top_fit_grams[:500]

[(('many', 'compliments'), 3278),
 (('true', 'size'), 3022),
 (('great', 'dress'), 1737),
 (('received', 'many'), 1481),
 (('got', 'many'), 1273),
 (('beautiful', 'dress'), 1242),
 (('fit', 'great'), 1207),
 (('perfect', 'dress'), 929),
 (('dress', 'comfortable'), 922),
 (('dress', 'great'), 889),
 (('fit', 'true'), 858),
 (('black', 'tie'), 850),
 (('great', 'fit'), 768),
 (('perfect', 'length'), 606),
 (('perfect', 'fit'), 603),
 (('little', 'tight'), 579),
 (('looked', 'great'), 542),
 (('able', 'wear'), 529),
 (('little', 'bit'), 510),
 (('easy', 'wear'), 498),
 (('comfortable', 'dress'), 497),
 (('gorgeous', 'dress'), 478),
 (('right', 'places'), 471),
 (('dress', 'little'), 431),
 (('fits', 'true'), 427),
 (('great', 'color'), 425),
 (('dress', 'gorgeous'), 401),
 (('beautiful', 'color'), 395),
 (('runs', 'true'), 395),
 (('little', 'short'), 376),
 (('felt', 'great'), 369),
 (('super', 'cute'), 355),
 (('low', 'cut'), 352),
 (('comfortable', 'wear'), 349),
 (('not', 'wear'), 347

In [329]:
top_small_grams[:20]

[(('runs', 'small'), 3019),
 (('many', 'compliments'), 2749),
 (('beautiful', 'dress'), 1856),
 (('great', 'dress'), 1734),
 (('received', 'many'), 1291),
 (('wear', 'size'), 1269),
 (('normally', 'wear'), 1160),
 (('little', 'tight'), 1155),
 (('little', 'small'), 1107),
 (('got', 'many'), 1001),
 (('bit', 'small'), 878),
 (('dress', 'great'), 841),
 (('black', 'tie'), 725),
 (('able', 'wear'), 714),
 (('gorgeous', 'dress'), 683),
 (('run', 'small'), 672),
 (('bit', 'tight'), 662),
 (('perfect', 'dress'), 651),
 (('ran', 'small'), 628),
 (('little', 'snug'), 626)]

In [331]:
top_large_grams[:20]

[(('many', 'compliments'), 2367),
 (('runs', 'large'), 2165),
 (('little', 'big'), 1701),
 (('great', 'dress'), 1561),
 (('beautiful', 'dress'), 1295),
 (('little', 'large'), 1218),
 (('received', 'many'), 999),
 (('got', 'many'), 973),
 (('smaller', 'size'), 935),
 (('wear', 'size'), 917),
 (('bit', 'large'), 885),
 (('normally', 'wear'), 877),
 (('runs', 'little'), 826),
 (('runs', 'big'), 767),
 (('dress', 'comfortable'), 767),
 (('dress', 'great'), 746),
 (('bit', 'big'), 737),
 (('perfect', 'dress'), 643),
 (('able', 'wear'), 632),
 (('fit', 'great'), 578)]

In [262]:
test = set(fit_ngram) & set(small_ngram) & set(large_ngram)

NameError: name 'fit_ngram' is not defined

 # Data Prediction

In [30]:
random.shuffle(data_all)

In [31]:
data_size = len(data_all)
valid_percent = 0.2
test_percent = 0.2


In [32]:
data_train = data_all[: int(data_size * (1 - valid_percent - test_percent))]
data_valid = data_all[
    int(data_size * (1 - valid_percent - test_percent)) : int(
        data_size * (1 - test_percent)
    )
]
data_test = data_all[int(data_size * (1 - test_percent)) :]


In [33]:
def extract_review(data):
    fit_reviews = [
        d["review_text"] + " " + d["review_summary"] for d in data if d["fit"] == "fit"
    ]
    small_reviews = [
        d["review_text"] + " " + d["review_summary"]
        for d in data
        if d["fit"] == "small"
    ]
    large_reviews = [
        d["review_text"] + " " + d["review_summary"]
        for d in data
        if d["fit"] == "large"
    ]
    return fit_reviews, small_reviews, large_reviews



 Try n-gram BoW logistic regression

In [34]:
n = 1
fit_reviews_train, small_reviews_train, large_reviews_train = extract_review(data_train)


In [35]:
top_fit_grams, top_small_grams, top_large_grams = top_adj_pipeline(
    fit_reviews_train, small_reviews_train, large_reviews_train, n
)


time consume: 8.67733097076416s


In [36]:
threshold = 500
top_word_set = {word for word, _ in top_fit_grams[:threshold]}.union(
    {word for word, _ in top_small_grams[:threshold]}.union(
        {word for word, _ in top_large_grams[:threshold]}
    )
)
wordId = dict(zip(top_word_set, range(len(top_word_set))))

In [37]:
def feature(d, n=1):
    feat = [0] * len(top_word_set)
    review = d["review_text"] + " " + d["review_summary"]
    p_list = as2_analysis.tokenize_paragraph(review, n=1)
    for word in p_list:
        if word not in top_word_set:
            continue
        # use BoW for now
        feat[wordId[word]] += 1
    return feat



In [38]:
def encode_output(data):
    return [0 if d["fit"] == "fit" else 1 if d["fit"] == "small" else 2 for d in data]



In [39]:
FIT = 0
SMALL = 1
BIG = 2
X_train = [feature(d, n) for d in data_train]
y_train = encode_output(data_train)
X_valid = [feature(d, n) for d in data_valid]
y_valid = encode_output(data_valid)


In [40]:
model = linear_model.LogisticRegression(C=1000)

In [41]:
model.fit(X_train, y_train)
y_pred = model.predict(X_valid)

In [42]:
np.mean(y_pred == y_valid)


0.758498013451401